In [ ]:
import pandas as pd
# Load the dataset
data = pd.read_csv("Allergen_Status_of_Food_Products.csv")

# Display the first few rows of the dataset for exploration
data.head()


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model

In [ ]:

# Preprocessing the data
data = pd.read_csv("Allergen_Status_of_Food_Products.csv")

# Fill NaN values with empty strings in the relevant columns
data = data.fillna('')

# Encoding the input features using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Food Product'] + " " + 
                             data['Main Ingredient'] + " " + 
                             data['Sweetener'] + " " + 
                             data['Fat/Oil'] + " " + 
                             data['Seasoning'])

# Preparing the target variable for multi-label classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['Allergens'].str.split(', '))

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Create the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
# Convert the sparse matrix to a dense NumPy array
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Train the model with the dense data
history = model.fit(X_train_dense, y_train, epochs=800, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set using the dense data
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Make predictions using the dense data
predictions = model.predict(X_test_dense)
model.save('ALE.h5')  # creates a HDF5 file 'my_model.h5'




In [ ]:
trained_model = load_model('ALE.h5')

In [ ]:
def predict_allergens_with_user_input(model, vectorizer, mlb, threshold=0.5):
    """
    Predicts allergens in a given food product based on user inputted ingredients
    and provides the likelihood of each allergen.

    Parameters:
    - model: Trained machine learning model for prediction.
    - vectorizer: CountVectorizer fitted on the training data.
    - mlb: MultiLabelBinarizer fitted on the training data.
    - threshold: Threshold for predicting the presence of an allergen (default is 0.5).

    Returns:
    - A dictionary with allergens and their likelihood.
    """

    # User input
    food_product = input("Enter Food Product Name: ")
    main_ingredient = input("Enter Main Ingredient: ")
    sweetener = input("Enter Sweetener (or None): ")
    fat_oil = input("Enter Fat/Oil (or None): ")
    seasoning = input("Enter Seasoning (or None): ")

    # Combining the input data and transforming it using the vectorizer
    combined_input = vectorizer.transform([f"{food_product} {main_ingredient} {sweetener} {fat_oil} {seasoning}"])

    # Getting model predictions
    pred_probabilities = model.predict(combined_input.toarray())[0]

    # Creating a dictionary of allergen probabilities
    allergen_probabilities = {allergen: prob for allergen, prob in zip(mlb.classes_, pred_probabilities)}

    # Filtering to include only allergens with probability above the threshold
    likely_allergens = {allergen: prob for allergen, prob in allergen_probabilities.items() if prob > threshold}
    if likely_allergens:
        return likely_allergens
    else:
        return "Does not contain allergens"

allergen_predictions = predict_allergens_with_user_input(trained_model, vectorizer, mlb)
allergen_predictions
